# Cost Reduction

We now see that solar can save a lot on grid consumption, but what we really care about is how much money it'll save.  In this notebook, we'll explore the differences of cost with and without solar.  We'll continue with the processed data from the previous notebook.

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

usage_data = pd.read_csv('../data/processed/Solar ROI - usage_data_with_generation.csv', parse_dates=['date'])
usage_data['start_time'] = pd.to_datetime(usage_data['start_time'], format='%H:%M:%S').dt.time

## Time of Use Calculation

We're currently on a the E-TOU-C plan, which is the [Residential Time-of-Use Peak Pricing 4-9 p.m. Every Day](https://www.pge.com/en/account/rate-plans/find-your-best-rate-plan/time-of-use-rate-plans.html#ResTOUplans-item-b206ca04d2) plan.  The structure for energy billing is as follows:

For the months October through May:
- The baseline is 11 kWh per day; i.e. the first 11 kWh per day are billed at 0.31976 and then 0.40827 thereafter.
- However, during the hours from 16:00 to 21:00 each day, the rates are 0.34811 and 0.43662 respectively.

For the months of June through September:
- The baseline is 9.8 kWh per day; i.e. the first 9.8 kWh per day are billed at 0.34811 and then 0.45589 thereafter.
- However, during the hours from 16:00 to 21:00 each day, the rates are 0.45082 and 0.53933 respectively.

We'll need to model the rate schedule to calculate our expected costs.

In [2]:
def calculate_tou_cost(row):
    date, month = row['date'], row['date'].month
    start_time, usage = row['start_time'], row['usage']

    # Define peak pricing hours
    peak_start_time = pd.to_datetime("16:00").time()
    peak_end_time = pd.to_datetime("21:00").time()

    # Adjust threshold based on time of year
    if month in [1, 2, 3, 4, 5, 10, 11, 12]:  # Jan-May, Oct-Dec
        threshold = 11

        # Adjust rate details for Tier 1 and Tier 2 during peak hours
        if peak_start_time <= start_time <= peak_end_time:
            rate_tier1 = 0.34811
            rate_tier2 = 0.43662
        else:  # Off-peak hours
            rate_tier1 = 0.31976
            rate_tier2 = 0.40827
    else:  # Jun-Sep
        threshold = 9.8

        # Adjust rate details for Tier 1 and Tier 2 during peak hours
        if peak_start_time <= start_time <= peak_end_time:
            rate_tier1 = 0.45082
            rate_tier2 = 0.53933
        else:  # Off-peak hours
            rate_tier1 = 0.36738
            rate_tier2 = 0.45589

    # Calculate cost based on tiers and threshold
    if usage <= threshold:
        cost = usage * rate_tier1
    else:
        cost = threshold * rate_tier1 + (usage - threshold) * rate_tier2

    return cost

# Create a new column 'cost_tou' to store the cost for each time window
tqdm.pandas(desc="Calculating TOU costs")
usage_data['cost_tou'] = usage_data.progress_apply(calculate_tou_cost, axis=1)

Calculating TOU costs:   0%|          | 0/35136 [00:00<?, ?it/s]